In [1]:
DATABASE='googol'
HOST='127.0.0.1'
USERNAME='kenjiwang'
PASSWORD='5517007'
PORT='5432'

In [2]:
import csv
import requests
import lxml.html

from angle_emb import AnglE, Prompts
from numpy.linalg import norm
import numpy as np
import psycopg2

/Users/kenjiwang/anaconda3/envs/search-engine/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/kenjiwang/anaconda3/envs/search-engine/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
conn = psycopg2.connect(database=DATABASE,
	host=HOST,
	user=USERNAME,
	password=PASSWORD,
	port=PORT
)

cursor = conn.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS websites (id bigserial PRIMARY KEY NOT NULL, url text NOT NULL, title text NOT NULL, description text NOT NULL, title_emb vector(1024), description_emb vector(1024))")

conn.commit()

In [4]:
def cosine_similarity(a,b):
	return np.sum(a*b, axis=1)/(norm(a, axis=1)*norm(b, axis=1))

In [5]:
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
# angle.set_prompt(prompt=Prompts.C)

In [6]:
def index_page(i, domain):
	if('api' in domain): return
	
	print(f"{i}: Indexing {domain}")
	website = requests.get('https://'+domain)
	if 'text/html' not in website.headers['content-type']: return
	html = lxml.html.fromstring(website.text)

	title = html.find('.//title').text
	title_formatted = title.replace('\'', '\'\'')
	title_emb = angle.encode(title, to_numpy=True)[0].tolist()

	try:
		try:
			description = html.find('.//meta[@name="description"]').get("content")
		except:
			description = html.find('.//meta[@property="og:description"]').get("content")
	except:
		description = ""

	description_emb = angle.encode(description, to_numpy=True)[0].tolist()
	description_formatted = description.replace('\'', '\'\'')

	query = f"INSERT INTO websites(title, url, description, title_emb, description_emb) VALUES ('{title_formatted}', '{domain}', '{description_formatted}', '{title_emb}', '{description_emb}')"
	cursor.execute(query)
	conn.commit()
	

In [7]:
with open('domains.csv', mode ='r') as file:
	csvFile = csv.reader(file)
	
	next(csvFile)
	for i in range(0,200): index_page(i, next(csvFile)[1])


0: Indexing facebook.com
2: Indexing google.com
3: Indexing youtube.com
4: Indexing twitter.com
5: Indexing googletagmanager.com
6: Indexing instagram.com
7: Indexing linkedin.com
8: Indexing fonts.gstatic.com
9: Indexing gmpg.org
10: Indexing maps.google.com
12: Indexing youtu.be
13: Indexing cdnjs.cloudflare.com
14: Indexing play.google.com
15: Indexing github.com
16: Indexing support.google.com
17: Indexing plus.google.com
18: Indexing en.wikipedia.org
19: Indexing drive.google.com
20: Indexing docs.google.com
21: Indexing wordpress.org
22: Indexing pinterest.com
23: Indexing goo.gl
24: Indexing developers.google.com
25: Indexing bit.ly
26: Indexing policies.google.com
27: Indexing amazon.com
28: Indexing cloudflare.com
29: Indexing google-analytics.com
30: Indexing vimeo.com
31: Indexing vk.com
32: Indexing itunes.apple.com
33: Indexing secure.gravatar.com
34: Indexing creativecommons.org
35: Indexing tiktok.com
36: Indexing medium.com
37: Indexing apps.apple.com
38: Indexing open.

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.